# Transform

Transfom represent kinds of transform matrixes, including translation, scaling, rotation and projection.

## Identity

In [ ]:
from py3d import Transform
Transform()


In [ ]:
assert _.sum() == 4


In [ ]:
Transform(n=(2,))


In [ ]:
assert _.sum() == 8


## Translation

### Translation Matrix

In **py3d**, transformations are represented as left-multiplication matrixes, point are represented as row vectors.

#### Left-multiplication translation matrix

In [ ]:
from sympy import symbols, Matrix
x, y, z, dx, dy, dz = symbols("x y z dx dy dz")
point = Matrix([x, y, z, 1]).T
translation = Matrix([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [dx, dy, dz, 1]
])
translation


In [ ]:
point * translation


#### Right-multiplication translation matrix

In [ ]:
translation.T


In [ ]:
translation.T * point.T


In [ ]:
from py3d import Vector3
Vector3([2, 3, 4]).as_translation()


In [ ]:
import numpy
assert numpy.array_equal(_[3, 0:3], [2, 3, 4])


In [ ]:
Transform.from_translation(x=1, n=(2,))


Translate a series of points

In [ ]:
import py3d
v = py3d.Viewer()
points = py3d.Vector3.Rand(100)
v.render(points.as_point())
points @= py3d.Transform.from_translation(x=1)
v.render(points.as_point())
v.show()


Move a car

In [ ]:
import py3d
v = py3d.Viewer()
car = py3d.Utils.Car()
grid = py3d.Utils.Grid(10)
t = 0
while t < 4:
    car.vertex @= py3d.Transform.from_translation(x=0.2)
    v.render(grid, car, t=t)
    t += 0.1
v.show()


## Scaling

In [ ]:
from py3d import Vector3
Vector3(x=1, y=2, z=(1, 2, 3)).as_scaling()


In [ ]:
Transform.from_scaling(x=1, y=-1, z=(1, 2, 3))


In [ ]:
from py3d import Vector3, Transform
points = (Vector3.Rand(1000)-0.5).U @ Transform.from_scaling(x=2)
points.as_point().render()


## Rotation

### Euler angles

#### Extrinsic Rotation

In [ ]:
import py3d
py3d.Transform.from_euler("xyz", [0.4, -0.2, 0])


#### Intrinsic Rotation

In [ ]:
import py3d
py3d.Transform.from_euler("XYZ", [0.4, -0.2, 0])


#### Difference between extrinsic rotation and intrinsic rotation

In [ ]:
import py3d
v = py3d.Viewer()
e_car = py3d.Utils.Car()
e_car.color = py3d.Color(g=1)
e_car.vertex @= py3d.Transform.from_euler("xyz", [0.4, -0.2, 0])
i_car = py3d.Utils.Car()
i_car.color = py3d.Color(r=1)
i_car.vertex @= py3d.Transform.from_euler("XYZ", [0.4, -0.2, 0])
v.render(e_car, i_car, py3d.Utils.Grid())
v.show()


#### RPY

In **py3d**, rpy represents roll, pitch and yaw, it is kind of intrinsic rotation

In [ ]:
from PIL import Image
import numpy
import py3d
img = numpy.array(Image.open("py3d/doc/20220917214012.jpg"))
hw = img.shape[0]//2
hh = img.shape[1]//2
point = py3d.Vector3.Grid(range(-hw, hw), range(-hh, hh))[::3, ::3]*0.01
point = point.as_point()
point.color = py3d.Vector3(img/255).H[::3, ::3]
point.vertex @= py3d.Transform.from_rpy([0, 0, -py3d.pi/2])
point.render()


### Quaternion

In **py3d**, quaternion is in scalar-last format

In [ ]:
import py3d
q = [
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0]
]
py3d.Transform.from_quaternion(q)


In [ ]:
from scipy.spatial.transform import Rotation
import numpy
assert numpy.allclose(_.as_quaternion(), q)
assert numpy.allclose(_[..., 0:3, 0:3], Rotation.from_quat(q).as_matrix())


In [ ]:
import py3d
py3d.Vector4(q).as_transform()


In [ ]:
from scipy.spatial.transform import Rotation
import numpy
assert numpy.allclose(_.as_quaternion(), q)
assert numpy.allclose(_[..., 0:3, 0:3], Rotation.from_quat(q).as_matrix())


### Convert RPY to quaternion

Return quaternion with sequence "xyzw" 

In [ ]:
import py3d
rpy = [
    [0, 0, -py3d.pi],
    [0, 0, -py3d.pi-0.01],
    [0, 0, -py3d.pi/3],
    [-1, 0, 0],
    [-py3d.pi, 0, 1],
    [0, -py3d.pi, 0]]
py3d.Transform.from_rpy(rpy).as_quaternion()


In [ ]:
import numpy
from scipy.spatial.transform import Rotation
q = Rotation.from_euler("XYZ", rpy).as_quat()
equal = numpy.all(numpy.isclose(q, _), axis=1)
opposite = numpy.all(numpy.isclose(-q, _), axis=1)
assert numpy.all(equal | opposite)


In [1]:
import py3d
# positions
p = [[3.24544068, -1.4610586,  1.21331756],
     [3.66378017, -1.32072563,  1.89712674],
     [-5.0884622, -2.48626808,  1.68773464],
     [-5.47338134, -2.65634697,  2.02463642],
     [1.830746, -0.8155359,  1.90245186],
     [-0.7094184, -0.84719837,  1.4467056],
     [-1.72178753, -0.681502,  1.17706321],
     [-3.88463547, -1.20610078,  1.14303617],
     [-4.527405, -3.12150274,  0.94426914],
     [4.13260871, -1.71061771,  1.49295544],
     [3.25896384, -1.46451182,  0.9032174],
     [-3.63891521, -1.03317465,  1.11405222]]
# quaternions
q = [[0.00307048, -0.27852711, -0.24115858,  0.92965357],
     [0.00955487, -0.328664, -0.25972646,  0.90798174],
     [0.05519327,  0.22308439,  0.22751421,  0.94626864],
     [0.05064761,  0.24596963,  0.23314524,  0.93945572],
     [-0.01006987, -0.17448035, -0.11591101,  0.97776267],
     [0.01230182, -0.03710485,  0.00123949,  0.99923489],
     [0.02991609,  0.0383105,  0.0729396,  0.99615117],
     [0.05252438,  0.12527874,  0.1242716,  0.98290538],
     [-0.09333274,  0.14651227,  0.2808575,  0.94389735],
     [0.00967634, -0.29085732, -0.28211318,  0.91417752],
     [0.00214324, -0.25691119, -0.23230781,  0.93809655],
     [0.04813863,  0.1177435,  0.11584668,  0.98508816]]

vertice = py3d.Vector3(z=[0, 1]) @ py3d.Transform.from_quaternion(q) @ py3d.Transform.from_translation(p)
directions = vertice.as_linesegment()
directions.start.color = py3d.Color(b=1)
directions.end.color = py3d.Color(r=1)
v = py3d.Viewer()
v.render(directions, py3d.Utils.Grid())
v.show()


Welcome to py3d world, please visit https://tumiz.github.io/scenario/ for more information


### Axis angles

In [ ]:
import py3d
py3d.Transform.from_angle_axis([2, 1], [0, 1, 0])


## Orthographic projection

In [ ]:
from sympy import Matrix, symbols
l, r, t, b, n, f = symbols("l r t b n f")
m_scale = Matrix([
    [2/(r-l), 0, 0, 0],
    [0, 2/(t-b), 0, 0],
    [0, 0, 2/(n-f), 0],
    [0, 0, 0, 1]
])
m_translate = Matrix([
    [1, 0, 0, -(l+r)/2],
    [0, 1, 0, -(b+t)/2],
    [0, 0, 1, -(f+n)/2],
    [0, 0, 0, 1]
])
m_scale*m_translate


## Perspective projection

In [ ]:
from py3d import Transform, pi, Vector3
projection = Transform.from_perspective(pi/2, 1.2, 0, 1000)
projection


In [ ]:
from py3d import Camera, pi, Vector3, Transform
import matplotlib.pyplot as plt
camera = Camera()
camera.transform.translation = Transform.from_translation([0, 0, 10])
camera.transform.rotation = Transform.from_rpy([0, 0, 0])
camera.projection = Transform.from_perspective(pi/3, 1, 1, 1000)
points = Vector3.Rand(100)
proj = points @ camera.matrix
plt.gca().set_aspect(1)
plt.scatter(proj.x, proj.y)


In [ ]:
from numpy import allclose
from py3d import Transform, Vector3
assert (Transform([
    [1., 0., 0.3, 0.],
    [0., 1., 0., 0.],
    [0., 0., 0.1, 0.],
    [-1., 0., 0., 1.]
]) == [
    [1., 0., 0.3, 0.],
    [0., 1., 0., 0.],
    [0., 0., 0.1, 0.],
    [-1., 0., 0., 1.]
]).all()
assert Transform(n=(2, 3)).shape == (2, 3, 4, 4)
assert Transform(n=(4, 5)).n == (4, 5)
assert ((Vector3(x=1).H @ Transform())[..., 0:3] == Vector3(x=1)).all()
assert (Vector3(x=1) @ Transform() == Vector3(x=1)).all()
p = Vector3([1, 2, 3])
T = Transform.from_translation([1, 2, 3])
R = Transform.from_euler("xyz", [0, 0, 1])
assert allclose(p @ T @ R, p @ (T@R))
